In [11]:
# Misc and Fundamentals
import pandas as pd
import matplotlib.pyplot as plt
import os
import joblib

# Setup 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Models
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn import ensemble

# Evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [12]:
tv_df = pd.read_csv("../Data/data/streaming_titles_clean.csv")
# Convert columns to dummies
print(tv_df["rating"].unique())
tv_df = pd.get_dummies(tv_df,columns=["country","rating"])

tv_df.head(3)

[nan 'PG-13' 'R' 'TV-14' 'PG' 'TV-PG' 'NOT RATED' 'TV-MA' 'G' 'TV-G'
 'TV-Y7' 'TV-Y' 'NR' '13+' '18+' '16+' 'TV-NR' 'ALL' '7+' 'NC-17'
 'UNRATED' 'AGES_16_' 'NOT_RATE' 'TV-Y7-FV' 'UR']


,type,title,director,cast,release_year,duration,description,score,imdbid,genre.Crime,...,rating_TV-14,rating_TV-G,rating_TV-MA,rating_TV-NR,rating_TV-PG,rating_TV-Y,rating_TV-Y7,rating_TV-Y7-FV,rating_UNRATED,rating_UR
0,Movie,Silent Night,NaN,NaN,2020,94.0,"Mark, a low end South London hitman recently r...",56.0,tt11628854,True,...,0,0,0,0,0,0,0,0,0,0
1,Movie,The Marksman,NaN,NaN,2021,108.0,A hardened Arizona rancher tries to protect an...,57.0,tt6902332,False,...,0,0,0,0,0,0,0,0,0,0
2,Movie,Gaia,NaN,NaN,2021,97.0,A forest ranger and two survivalists with a cu...,63.0,tt11881160,False,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Train/Test split
tv_train, tv_test = train_test_split(tv_df, random_state = 2023) # random state important for evaluation

X = tv_train.loc[:,tv_train.columns != "score"]
y = tv_train["score"]

The following function is to be able to save and load models without having to re-compute everything every time.

In [14]:
def fitmodel(model, filename, df = tv_train):
    
    if not os.path.isfile(filename):

        model.fit(X,y)

        joblib.dump(model, filename)

    else:
        modeltemp = joblib.load(filename)
        if (type(model) != type(modeltemp)) or \
            (tuple([k[0] for k in model.steps]) != tuple([k[0] for k in modeltemp.steps])):
            print ("\033[93m Warning: model mismatch. Delete the file {filename} and rerun or risk faulty models.\n \033[0m".format(filename=filename))
        model = modeltemp
    
    return model

In [15]:
from sklearn.metrics import get_scorer_names
# get_scorer_names()[20:]

We are going to use the `neg_root_mean_squared_error` metric, because it essentially gives us the weighted distance our model is from the correct metric.

In [16]:
from numpy import mean, std
print("mean: %0.1f, rMSE: %0.4f"% (mean(y.values),std(y.values)))

mean: 48.4, rMSE: 21.7000


A rMSE of 21.7 here means that if we guessed the mean every time, we would be on average off by 21.7 points. So, our model should try to get a better rMSE than 21.7.

In [26]:
from ModelHelpers import *

Let's start by making a Beta Regression predictor.

In [27]:
beta_pipe = Pipeline(steps =  [
    ("predictors", ColumnSelector(columns=["release_year"],
                                  startswith=["genre","rating"])),
    ("beta_regression", BetaRegression(from_range=(0,100)))
    ])

tv_folded = KFold(n_splits = 5).split(X,y)

lm_cross_val = cross_val_score(estimator = beta_pipe,
                               cv = tv_folded,
                               scoring = 'neg_root_mean_squared_error',
                               X = X,
                               y = y,
                               verbose = 0)

print("%0.4f root MSE with a standard deviation of %0.4f." % (-1*lm_cross_val.mean(), lm_cross_val.std()))

19.7978 root MSE with a standard deviation of 0.2440.


In [28]:
fitmodel(beta_pipe, "models/beta_regression.joblib")
pass